In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import unidecode
import json

# Caso hipotético
## A un yucateco le sobran y estorban unos 300 mil pesos, y tiene ganas de invertirlos en un negocio de comida, pero quiere saber en cuál conviene.


## Lo primero que hacemos es conocer la oferta existente, lo cual revisamos de las bases de datos de Ingegi.

Obtenemos archivo CSV de la siguiente liga:
https://inegi.org.mx/app/descarga/default.html

# Inegi

In [2]:
inegi = pd.read_csv('./conjunto_de_datos/denue_inegi_72_2.csv', encoding='ISO-8859-1')

/home/linuxbrew/.linuxbrew/Cellar/ipython/7.8.0/libexec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
inegi.columns

Index(['id', 'nom_estab', 'raz_social', 'codigo_act', 'nombre_act', 'per_ocu',
       'tipo_vial', 'nom_vial', 'tipo_v_e_1', 'nom_v_e_1', 'tipo_v_e_2',
       'nom_v_e_2', 'tipo_v_e_3', 'nom_v_e_3', 'numero_ext', 'letra_ext',
       'edificio', 'edificio_e', 'numero_int', 'letra_int', 'tipo_asent',
       'nomb_asent', 'tipoCenCom', 'nom_CenCom', 'num_local', 'cod_postal',
       'cve_ent', 'entidad', 'cve_mun', 'municipio', 'cve_loc', 'localidad',
       'ageb', 'manzana', 'telefono', 'correoelec', 'www', 'tipoUniEco',
       'latitud', 'longitud', 'fecha_alta'],
      dtype='object')

In [4]:
inegi['entidad'].value_counts()

VERACRUZ DE IGNACIO DE LA LLAVE             48459
YUCATÁN                                     17180
SAN LUIS POTOSÍ                             15655
TAMAULIPAS                                  15576
SINALOA                                     15321
SONORA                                      13198
QUERÉTARO                                   12448
TABASCO                                     11959
QUINTANA ROO                                11275
ZACATECAS                                    8474
TLAXCALA                                     8267
Name: entidad, dtype: int64

### Como el negocio se busca ubicar en Mérida , Yucatán, filtramos por estado y luego por localidad.

In [5]:
inegi['entidad'] = inegi['entidad'].apply(lambda x: 'YUCATAN' if 'YUCA' in x else x)
yinegi = inegi[inegi['entidad']=='YUCATAN']
yinegi.head()

,id,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,...,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,latitud,longitud,fecha_alta
152158,4512197,100 ARRACHERO,NaN,722514,Restaurantes con servicio de preparación de ta...,6 a 10 personas,CALLE,21,CALLE,10,...,Mérida,5244,20.0,9.99938e+09,NaN,NaN,Fijo,21.001433,-89.605708,2010-07
152159,7688466,100 MONTADITOS,MONTADITOSMERIDA SA DE CV,722511,Restaurantes con servicio de preparación de al...,11 a 30 personas,CALLE,11,CALLE,20A,...,Mérida,6543,11.0,9.99368e+09,MONTADITOSMERIDA@GMAIL.COM,NaN,Fijo,21.038795,-89.601918,2019-11
152160,4572894,100 NATURAL,"ALIMENTOS SALUD Y BIENESTAR, S.A. DE C.V.",722515,"Cafeterías, fuentes de sodas, neverías, refres...",11 a 30 personas,CALLE,1,AVENIDA,50 DIAGONAL (PROLONGACIÓN PASEO DE MONTEJO),...,Mérida,1453,19.0,NaN,NaN,NaN,Fijo,21.024276,-89.624382,2010-07
152161,4536636,100 NATURAL RESTAURANTES,OPERADORA AQSE SA DE CV,722511,Restaurantes con servicio de preparación de al...,11 a 30 personas,CALLE,22,CALLE,19,...,Mérida,5598,15.0,9.9913e+08,NaN,WWW.100NATURAL.COM,Fijo,21.020646,-89.585322,2014-12
152162,7757564,100 PIZZAS,NaN,722517,Restaurantes con servicio de preparación de pi...,11 a 30 personas,CALLE,35,CALLE,12,...,Mérida,3695,15.0,9.99737e+09,ACEVEDOTELLOCARLOS@GMAIL.COM,NaN,Fijo,20.998787,-89.566374,2019-11


In [6]:
yinegi['localidad'].value_counts()

Mérida                                  7475
Valladolid                               636
Kanasín                                  600
Tizimín                                  529
Ticul                                    483
                                        ... 
Sotuta de Peón                             1
San Miguel                                 1
Maximiliano de Jesús Magaña González       1
El Cuyo                                    1
Misión [Hotel]                             1
Name: localidad, Length: 157, dtype: int64

In [7]:
yinegi.drop(yinegi[yinegi['localidad']!='Mérida'].index, inplace=True)
yinegi['localidad'].value_counts()

todrop = ['cve_ent', 'entidad', 'codigo_act', 'cve_mun', 'municipio', 'cve_loc', 'localidad', 'ageb', 'nom_vial', 'tipo_v_e_1', 'nom_v_e_1', 'tipo_v_e_2', 'nom_v_e_2', 'tipo_v_e_3', 'nom_v_e_3', 'numero_ext', 'letra_ext', 'edificio', 'edificio_e', 'numero_int', 'letra_int', 'tipo_asent', 'nomb_asent', 'tipoCenCom', 'nom_CenCom', 'num_local','ageb', 'manzana', 'telefono', 'correoelec', 'www']
yinegi.drop(columns = todrop, inplace=True)

/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Eliminamos aquellas filas en los que los negocios no describan su mercado.

In [51]:
yinegi.drop(yinegi[yinegi['nom_estab'].isnull()].index, inplace=True)

In [52]:
yinegi.shape

(4772, 11)

In [9]:
yinegi['nombre_act'].value_counts()

Restaurantes que preparan otro tipo de alimentos para llevar                                                  1483
Restaurantes con servicio de preparación de antojitos                                                         1339
Restaurantes con servicio de preparación de tacos y tortas                                                    1154
Restaurantes con servicio de preparación de pizzas, hamburguesas, hot dogs y pollos rostizados para llevar     737
Restaurantes con servicio de preparación de alimentos a la carta o de comida corrida                           692
Cafeterías, fuentes de sodas, neverías, refresquerías y similares                                              630
Servicios de preparación de otros alimentos para consumo inmediato                                             312
Bares, cantinas y similares                                                                                    240
Restaurantes de autoservicio                                                    

### Eliminamos aquellos negocios que no tienen qué ver con alimentos.

In [10]:
yinegi.drop(yinegi[yinegi['nombre_act']=='Cafeterías, fuentes de sodas, neverías, refresquerías y similares'].index, inplace=True)
yinegi.drop(yinegi[yinegi['nombre_act']=='Bares, cantinas y similares'].index, inplace=True)
yinegi.drop(yinegi[yinegi['nombre_act']=='Bares, cantinas y similares'].index, inplace=True)
yinegi.drop(yinegi[yinegi['nombre_act']=='Hoteles sin otros servicios integrados'].index, inplace=True)
yinegi.drop(yinegi[yinegi['nombre_act']=='Hoteles con otros servicios integrados'].index, inplace=True)
yinegi.drop(yinegi[yinegi['nombre_act']=='Servicios de preparación de alimentos para ocasiones especiales'].index, inplace=True)
yinegi.drop(yinegi[yinegi['nombre_act']=='Pensiones y casas de huéspedes'].index, inplace=True)
yinegi.drop(yinegi[yinegi['nombre_act']=='Moteles'].index, inplace=True)
yinegi.drop(yinegi[yinegi['nombre_act']=='Centros nocturnos, discotecas y similares'].index, inplace=True)
yinegi.drop(yinegi[yinegi['nombre_act']=='Servicios de comedor para empresas e instituciones'].index, inplace=True)
yinegi.drop(yinegi[yinegi['nombre_act']=='Departamentos y casas amueblados con servicios de hotelería'].index, inplace=True)
yinegi['nombre_act'].value_counts()

Restaurantes que preparan otro tipo de alimentos para llevar                                                  1483
Restaurantes con servicio de preparación de antojitos                                                         1339
Restaurantes con servicio de preparación de tacos y tortas                                                    1154
Restaurantes con servicio de preparación de pizzas, hamburguesas, hot dogs y pollos rostizados para llevar     737
Restaurantes con servicio de preparación de alimentos a la carta o de comida corrida                           692
Servicios de preparación de otros alimentos para consumo inmediato                                             312
Restaurantes de autoservicio                                                                                   238
Restaurantes con servicio de preparación de pescados y mariscos                                                223
Servicios de preparación de alimentos en unidades móviles                       

In [11]:
yinegi.drop_duplicates(inplace=True)
yinegi.shape

/home/linuxbrew/.linuxbrew/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(6208, 11)

In [12]:
yinegi['nom_estab'].value_counts()

COCINA ECONÓMICA SIN NOMBRE               96
COCINA ECONÓMICA                          93
VENTA DE ANTOJITOS                        75
VENTA DE ANTOJITOS SIN NOMBRE             65
LONCHERÍA SIN NOMBRE                      46
                                          ..
COCINA ECONÓMICA Y PIZERIA LA GLORIETA     1
POLLERIA JACARANDOSO                       1
MIMI PANUCHOS Y SALBUTES                   1
KRAKEN                                     1
COCHINITA                                  1
Name: nom_estab, Length: 4755, dtype: int64

### Es necesario agrupas tipos de alimentos, por lo renombramos los valores para encasillarlos a grupos específicos.

In [13]:
yinegi['nom_estab'] = yinegi['nom_estab'].apply(lambda x: x.lower())

pd.set_option('display.max_rows', None)
yinegi['nom_estab'].value_counts()

/home/linuxbrew/.linuxbrew/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


cocina económica sin nombre                                                                                             96
cocina económica                                                                                                        93
venta de antojitos                                                                                                      75
venta de antojitos sin nombre                                                                                           65
lonchería sin nombre                                                                                                    46
venta de panuchos y salbutes                                                                                            41
lonchería                                                                                                               33
venta de comida sin nombre                                                                                              30
antojitos sin no

In [14]:
def getFoodType(name):
    if 'piz' in name or 'zz' in name:
        return 'pizza'
    elif 'cocina' in name or 'comida' in name:
        return 'cocina económica'
    elif 'antoj' in name:
        return 'antojitos'
    elif 'lonch' in name:
        return 'lonchería'
    elif 'tac' in name or 'taq' in name:
        return 'tacos'
    elif 'burg' in name:
        'hamburgesas'
    elif 'poll' in name:
        return 'pollo'
    elif 'cochin' in name:
        return 'cochinita'
    elif 'panuch' in name:
        return 'panuchos'
    elif 'carnitas' in name:
        return 'carnitas'
    elif 'tamal' in name:
        return 'tamales'
    elif 'tort' in name:
        return 'tortas'
    elif 'arrach' in name:
        return 'arracheras'
    elif 'pescad' in name or 'maris' in name or 'cevich' in name:
        return 'mariscos'
    elif 'desay' in name:
        return 'desayunos'
    elif 'asad' in name:
        return 'carne asada'
    elif 'chichar' in name:
        return 'chicharronería'
    elif 'dog' in name:
        return 'hot dogs'
    elif 'sushi' in name:
        return 'sushi'
    elif 'restauran' in name:
        return 'restaurante'
    else:
        return 'drop'

In [15]:
yinegi['nom_estab'] = yinegi['nom_estab'].apply(getFoodType)
    
yinegi['nom_estab'].value_counts()

/home/linuxbrew/.linuxbrew/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


cocina económica    1329
drop                1304
tacos                665
antojitos            506
lonchería            500
pizza                326
restaurante          325
pollo                217
panuchos             180
mariscos             127
tortas               112
cochinita            104
tamales               80
carne asada           71
desayunos             70
chicharronería        69
sushi                 38
carnitas              33
hot dogs              17
arracheras             3
Name: nom_estab, dtype: int64

### Eliminamos aquellos que con valor genérico 'drop'.

In [16]:
yinegi.drop(yinegi[yinegi['nom_estab']=='drop'].index, inplace=True)

In [17]:
tipos_comida = yinegi['nom_estab'].value_counts()
tipos_comida

cocina económica    1329
tacos                665
antojitos            506
lonchería            500
pizza                326
restaurante          325
pollo                217
panuchos             180
mariscos             127
tortas               112
cochinita            104
tamales               80
carne asada           71
desayunos             70
chicharronería        69
sushi                 38
carnitas              33
hot dogs              17
arracheras             3
Name: nom_estab, dtype: int64

In [18]:
yinegi.columns

Index(['id', 'nom_estab', 'raz_social', 'nombre_act', 'per_ocu', 'tipo_vial',
       'cod_postal', 'tipoUniEco', 'latitud', 'longitud', 'fecha_alta'],
      dtype='object')

,id,nom_estab,raz_social,nombre_act,per_ocu,tipo_vial,cod_postal,tipoUniEco,latitud,longitud,fecha_alta
152234,7991625,None,GRUPO EXPANSION BY RER SA,Restaurantes de autoservicio,6 a 10 personas,AVENIDA,97205.0,Fijo,21.013214,-89.652736,2019-11
152871,7689456,None,NaN,Servicios de preparación de alimentos en unida...,6 a 10 personas,CALLE,97147.0,Fijo,21.001108,-89.580396,2019-11
153197,4536409,None,,Restaurantes de autoservicio,6 a 10 personas,CALLE,97115.0,Fijo,21.029564,-89.626076,2014-12
153223,9024204,None,NaN,Restaurantes con servicio de preparación de pi...,0 a 5 personas,CALLE,97190.0,Fijo,20.939954,-89.601700,2019-11
153250,9206358,None,OPERADORA EXE SA DE CV,Restaurantes de autoservicio,0 a 5 personas,CALLE,97288.0,Fijo,20.927779,-89.684918,2019-11
153358,8352747,None,NaN,Restaurantes con servicio de preparación de pi...,0 a 5 personas,CALLE,97158.0,Fijo,20.976256,-89.598943,2019-11
153360,6795438,None,OPERADORA EXE SA DE CV,Restaurantes de autoservicio,11 a 30 personas,CALLE,97305.0,Fijo,21.016921,-89.582884,2017-03
153361,6795433,None,OPERADORA EXE SA DE CV,Restaurantes de autoservicio,11 a 30 personas,CALLE,97305.0,Fijo,21.015884,-89.583654,2017-03
153362,6415939,None,OPERADORA EXE SA DE CV,Restaurantes de autoservicio,11 a 30 personas,AVENIDA,97259.0,Fijo,20.947531,-89.655277,2011-03
153363,6852517,None,OPERADORA EXE SA DE CV,Restaurantes de autoservicio,11 a 30 personas,CALLE,97127.0,Fijo,20.998290,-89.616119,2018-11


###  Por último, enlistamos los principales giros de comida.

In [20]:
tipos_comida.index
tipos_comida = list(tipos_comida.index)
tipos_comida

['cocina económica',
 'tacos',
 'antojitos',
 'lonchería',
 'pizza',
 'restaurante',
 'pollo',
 'panuchos',
 'mariscos',
 'tortas',
 'cochinita',
 'tamales',
 'carne asada',
 'desayunos',
 'chicharronería',
 'sushi',
 'carnitas',
 'hot dogs',
 'arracheras']

### Con esta información, revisaremos qué tanto interés hay por estos giros de comida.

# Google Trends

### Revisaremos qué tantas búsquedas se hacen en Google sobre cada uno de los giros de comida, con el fin de conocer dónde está el interés del consumidor.
### Para esto utilizaremos la API de Google Trends a través de pytrends.

In [21]:
from pytrends.request import TrendReq

pytrends = TrendReq(hl='es-MX', tz=360)

In [22]:
resultados = []
for comida in tipos_comida:
    pytrends.build_payload([comida], cat=71, geo='MX', gprop='')
    resultados.append(pytrends.interest_by_region(resolution='REGION', inc_low_vol=True, inc_geo_code=False))
    
resultados

[                      cocina económica
 geoName                               
 Aguascalientes                       0
 Baja California                      0
 Baja California Sur                  0
 Campeche                             0
 Chiapas                              0
 Chihuahua                            0
 Ciudad de México                    22
 Coahuila de Zaragoza                 0
 Colima                               0
 Durango                              0
 Estado de México                    22
 Guanajuato                           0
 Guerrero                             0
 Hidalgo                              0
 Jalisco                              0
 Michoacán                            0
 Morelos                              0
 Nayarit                              0
 Nuevo León                           0
 Oaxaca                               0
 Puebla                               0
 Querétaro                            0
 Quintana Roo                         0


### Como los resultados nos los da por estados, filtramos y nos quedamos con las búsquedas en Yucatán.

In [23]:
busquedas = resultados[0].join(resultados[1:]).T

In [56]:
trends = busquedas['Yucatán'].sort_values(ascending=False)
trends.loc['tamales']

54

In [40]:
trends_dict['cochinita']

88

In [58]:
yinegi['trend'] = yinegi['nom_estab'].apply(lambda x: trends.loc[x])
yinegi[['nom_estab','trend']]

/home/linuxbrew/.linuxbrew/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,nom_estab,trend
152158,arracheras,34
152161,restaurante,39
152162,pizza,100
152163,arracheras,34
152171,restaurante,39
152179,cocina económica,100
152184,sushi,30
152195,pizza,100
152196,pollo,79
152205,pizza,100


### Con esto, tenemos parametrizado el interés en el estado por cada uno de los giros de comida.

### Ahora, necesitamos revisar el grado de satisfacción de los usuarios por la oferta existente.

# Yelp

### Utilizando la API de Yelp, hacemos búsquedas nuevamente sobre los diferentes giros de comida para encontrar negocios y sus calificaciónes en la plataforma.

In [59]:
client_id = 'JPrLnyol5AaT0XLf7CYbWg'
api_key = '5F0kTJmEIUsrAXKEcpqHA_HvL2uKWA3HZC_iQ73dGgIXgke7fCh84CBP5KpwVMXumfm4BahGo3iaBiNtBi9Q9vexfphH6urta05XSFaaQCs6aphzSQfWwhWSo_6nXHYx'
app_name = 'ravenous'

endpoint = 'https://api.yelp.com/v3/businesses/search?'
req_parameters_1 = ['location']
req_parameters_2 = ['latitud', 'longitude']
optional_parameters = ['term', 'radius', 'categories', 'locale', 'limit', 'sort_by', 'price']

In [63]:
yelp_search = []
for comida in tipos_comida:
    term = 'term={}&'.format(comida)
    print(term)
    location = 'latitude=21.073881&longitude=-89.655476&'
    locale = 'locale=es_MX&'
    categories = 'restaurants,foodstands'
    limit = 'limit=50'
    res = requests.get(endpoint+term+location+locale+limit, headers={'Authorization':'Bearer '+api_key})
    soup = bs(res.content)
    yelp_json = json.loads(soup.findAll('p')[0].text)
    ydf = pd.DataFrame(yelp_json['businesses'])
    ydf['tipo'] = comida
    yelp_search.append(ydf)
    
yelp_search

term=cocina económica&
term=tacos&
term=antojitos&
term=lonchería&
term=pizza&
term=restaurante&
term=pollo&
term=panuchos&
term=mariscos&
term=tortas&
term=cochinita&
term=tamales&
term=carne asada&
term=desayunos&
term=chicharronería&
term=sushi&
term=carnitas&
term=hot dogs&
term=arracheras&


[                        id                                              alias  \
 0   a1oMI_2Bll8V38pMmh3C5A                      cocina-económica-laura-mérida   
 1   u6quJ-v_REMlOl8qtlsVsQ               cocina-económica-altabrisa-altabrisa   
 2   oazpmGeb5m1hSv44oR_xsg     cocina-económica-y-cenaduría-el-remedio-mérida   
 3   gci7gop2fxTpmiUv26N34Q                      la-cocina-de-emilia-vergel-ii   
 4   2Qb0-JhFAC2VCZaTb3ok1w                  cocina-económica-los-pinos-mérida   
 5   9l4NrLx8bFKeieEdencPSg                             kitchen-by-mina-mérida   
 6   8ST_oRKKasGwvzDh9FOUag       taquería-y-cocina-económica-pensiones-mérida   
 7   hJehvLFO-XRVfNHo1gz20Q                             las-bugambilias-mérida   
 8   pF01k3BB2pKXndFCJm6heA                              las-sombrillas-mérida   
 9   bHuEeXA1XFbr1xu1-4s6GQ  cafetería-de-la-facultad-de-psicología-uady-al...   
 10  JaC7VgnA-KYGaIrXbmwASg                               cocina-y-algo-mérida   
 11  RqABoMit60P

In [64]:
lugares = pd.concat(yelp_search, axis=0, sort=False)
print(lugares.shape)

(733, 17)


In [66]:
lugares.tipo.value_counts()

pollo               50
desayunos           50
tacos               50
tortas              50
cocina económica    50
carne asada         50
pizza               50
arracheras          50
restaurante         50
mariscos            50
antojitos           38
cochinita           36
tamales             31
sushi               31
panuchos            30
carnitas            22
lonchería           21
hot dogs            18
chicharronería       6
Name: tipo, dtype: int64

In [67]:
lugares.reset_index(inplace=True)

### La información más relevante que extraemos de Yelp es el 'rating' por giro de comida, por lo que de los resultados sacamos promedios basados en su giro.

In [83]:
ratings = dict(lugares.tipo.value_counts())
for comida in tipos_comida:
    ratings[comida] = lugares[lugares.tipo == comida]['rating'].mean()
ratings

{'pollo': 4.13,
 'desayunos': 4.33,
 'tacos': 4.33,
 'tortas': 4.12,
 'cocina económica': 4.14,
 'carne asada': 4.06,
 'pizza': 4.08,
 'arracheras': 4.05,
 'restaurante': 4.5,
 'mariscos': 4.23,
 'antojitos': 4.026315789473684,
 'cochinita': 4.194444444444445,
 'tamales': 4.145161290322581,
 'sushi': 4.048387096774194,
 'panuchos': 3.9,
 'carnitas': 4.318181818181818,
 'lonchería': 4.0476190476190474,
 'hot dogs': 4.194444444444445,
 'chicharronería': 4.333333333333333}

# Análisis Final

### Ahora hace falta integrar toda la información.

### Primero revisamos la oferta con los datos del Inegi, y lo dejamos en términos porcentuales para conocer cómo están distribuidos los giros de comida.

In [84]:
yinegi['ratings'] = yinegi['nom_estab'].apply(lambda x: ratings[x])
yinegi.columns

/home/linuxbrew/.linuxbrew/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Index(['id', 'nom_estab', 'raz_social', 'nombre_act', 'per_ocu', 'tipo_vial',
       'cod_postal', 'tipoUniEco', 'latitud', 'longitud', 'fecha_alta',
       'trend', 'ratings'],
      dtype='object')

In [99]:
analysis = pd.DataFrame()
analysis['qty'] = yinegi['nom_estab'].value_counts()
analysis

,qty
cocina económica,1329
tacos,665
antojitos,506
lonchería,500
pizza,326
restaurante,325
pollo,217
panuchos,180
mariscos,127
tortas,112


In [100]:
analysis['percentage'] = analysis.qty.apply(lambda x: x*100/analysis.qty.sum())
analysis

,qty,percentage
cocina económica,1329,27.849958
tacos,665,13.935457
antojitos,506,10.603521
lonchería,500,10.477787
pizza,326,6.831517
restaurante,325,6.810562
pollo,217,4.547360
panuchos,180,3.772003
mariscos,127,2.661358
tortas,112,2.347024


### Luego, integramos la información de Yelp, sobre el grado de satisfacción con la oferta existente.

In [105]:
ratings

{'pollo': 4.13,
 'desayunos': 4.33,
 'tacos': 4.33,
 'tortas': 4.12,
 'cocina económica': 4.14,
 'carne asada': 4.06,
 'pizza': 4.08,
 'arracheras': 4.05,
 'restaurante': 4.5,
 'mariscos': 4.23,
 'antojitos': 4.026315789473684,
 'cochinita': 4.194444444444445,
 'tamales': 4.145161290322581,
 'sushi': 4.048387096774194,
 'panuchos': 3.9,
 'carnitas': 4.318181818181818,
 'lonchería': 4.0476190476190474,
 'hot dogs': 4.194444444444445,
 'chicharronería': 4.333333333333333}

In [116]:
analysis = pd.concat([analysis, pd.Series(ratings)], axis=1)

/home/linuxbrew/.linuxbrew/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [119]:
analysis.rename(columns = {0:'rating'}, inplace=True)
analysis

,qty,percentage,rating
antojitos,506,10.603521,4.026316
arracheras,3,0.062867,4.050000
carne asada,71,1.487846,4.060000
carnitas,33,0.691534,4.318182
chicharronería,69,1.445935,4.333333
cochinita,104,2.179380,4.194444
cocina económica,1329,27.849958,4.140000
desayunos,70,1.466890,4.330000
hot dogs,17,0.356245,4.194444
lonchería,500,10.477787,4.047619


### También integramos la información de Google Trends sobre el interés de los consumidores por cada uno de los giros de comida.

In [120]:
trends

cocina económica    100
panuchos            100
pizza               100
cochinita            88
pollo                79
desayunos            77
tamales              54
carnitas             42
restaurante          39
antojitos            37
arracheras           34
carne asada          32
mariscos             31
sushi                30
hot dogs             29
tacos                22
tortas               22
chicharronería        0
lonchería             0
Name: Yucatán, dtype: int64

In [121]:
analysis = pd.concat([analysis, trends], axis=1)

/home/linuxbrew/.linuxbrew/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [123]:
analysis.rename(columns = {'Yucatán': 'trend'}, inplace=True)
analysis

,qty,percentage,rating,trend
antojitos,506,10.603521,4.026316,37
arracheras,3,0.062867,4.050000,34
carne asada,71,1.487846,4.060000,32
carnitas,33,0.691534,4.318182,42
chicharronería,69,1.445935,4.333333,0
cochinita,104,2.179380,4.194444,88
cocina económica,1329,27.849958,4.140000,100
desayunos,70,1.466890,4.330000,77
hot dogs,17,0.356245,4.194444,29
lonchería,500,10.477787,4.047619,0


## Decisión 

### Para decidir el giro del negocio de comida, se toman las siguientes consideraciones:
#### 1. Se prefiere un giro con alto interés (trend).
#### 2. Se prefiere un giro con baja satisfacción por la población, por ser área de oportunidad. (rating)
#### 3. Conviene un giro con poca oferta, pues significa menor competencia. (percentage)

### Fórmula utilizada
$$score = \frac{trend}{20*rating}$$

In [141]:
analysis['score'] = analysis['trend'] / (100 * analysis['rating']/5)
analysis.sort_values('score', ascending=False).head()

,qty,percentage,rating,trend,score
panuchos,180,3.772003,3.900000,100,1.282051
pizza,326,6.831517,4.080000,100,1.225490
cocina económica,1329,27.849958,4.140000,100,1.207729
cochinita,104,2.179380,4.194444,88,1.049007
pollo,217,4.547360,4.130000,79,0.956416


### Indudablemente, conviene poner un restaurante de panuchos. Because yucas.